## Import modules

In [9]:
# Import modules
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore")


print("Packages installed")

Packages installed


## 

## Read CSV

In [10]:
dataframes=[]
for dirname, _, filenames in os.walk('../DataSets/RawData/'):
    for filename in filenames:
        file=filename.split('.')
        file=((file[0]+"_df"))
        if file !="_df":
            filepath=os.path.join(dirname,filename)
            df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")
            exec(f'{file} = df.copy()')
            print(file, df.shape)
            dataframes.append(df)
print('Data imported') #Tar ca. 10 sekunder å lese alle filene

appearances_df (1533437, 13)
clubs_df (426, 17)
club_games_df (133132, 11)
competitions_df (43, 10)
games_df (66566, 23)
game_events_df (687761, 10)
game_lineups_df (2319806, 9)
players_df (30482, 23)
player_valuations_df (465228, 5)
Data imported


## Date functions

In [11]:
#0.25 vårsesongen, 0.75 høstsesongen.
def date_to_year(df):
    df["year"] = pd.to_datetime(df["date"]).dt.year
    df["month"] = pd.to_datetime(df["date"]).dt.month

    df["year"] = np.where(df["month"] <= 6, df["year"] + 0.25, df["year"] + 0.75)
    return df

def valuation_date_to_year(df):
    df["year"] = pd.to_datetime(df["date"]).dt.year
    df["month"] = pd.to_datetime(df["date"]).dt.month

    conditions = [
    (df['month'] > 2) & (df['month'] < 9),
    (df['month'] >= 9),
    (df['month'] < 3)
    ]

    choices = ['Spring', 'Fall', 'Fall']
    df['semester'] = np.select(conditions, choices)
    df.loc[(df['month'] < 3), 'year'] -= 1
    df["year"] = np.where(df["semester"] == "Spring", df["year"] + 0.25, df["year"] + 0.75)
    
    return df

def calculate_age(row):
    date_of_birth = pd.to_datetime(row['date_of_birth'])
    date = pd.to_datetime(row['date'])
    age = date.year - date_of_birth.year - ((date.month, date.day) < (date.month, date.day))
    return age

## 1. Games & Appearances

In [12]:
games_df = games_df[["game_id", "date", "home_club_id", "away_club_id", "home_club_goals", "away_club_goals"]]
appearances_df = appearances_df[["game_id", "player_id", "player_club_id", "yellow_cards", "red_cards", "goals", "assists", "minutes_played"]]

games_df = pd.merge(games_df, appearances_df, on="game_id")

games_df['goals_for'] = games_df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] else row['away_club_goals'], axis=1)
games_df['goals_against'] = games_df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] else row['home_club_goals'], axis=1)
games_df = date_to_year(games_df)
games_df.head()

,game_id,date,home_club_id,away_club_id,home_club_goals,away_club_goals,player_id,player_club_id,yellow_cards,red_cards,goals,assists,minutes_played,goals_for,goals_against,year,month
0,2321044,2013-08-18,16,23,2.0,1.0,121404,23,0,0,0,0,90,1.0,2.0,2013.75,8
1,2321044,2013-08-18,16,23,2.0,1.0,125103,16,0,0,0,0,1,2.0,1.0,2013.75,8
2,2321044,2013-08-18,16,23,2.0,1.0,1659,23,0,0,0,0,77,1.0,2.0,2013.75,8
3,2321044,2013-08-18,16,23,2.0,1.0,16831,23,0,0,0,0,90,1.0,2.0,2013.75,8
4,2321044,2013-08-18,16,23,2.0,1.0,26,16,0,0,0,0,90,2.0,1.0,2013.75,8


## 2. Player performance

In [13]:
games_df = games_df[["player_id", "player_club_id", "yellow_cards", "red_cards", "goals", "assists", "minutes_played", "goals_for", "goals_against", "year"]]
player_performance_df = games_df.groupby(['player_id', 'player_club_id', 'year']).agg({
    'goals_for': 'sum',
    'goals_against': 'sum',
    'goals': 'sum', 
    'assists': 'sum', 
    'red_cards': 'sum', 
    'yellow_cards': 'sum', 
    'minutes_played': 'sum'
}).reset_index()

player_performance_df.head()

,player_id,player_club_id,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played
0,10,398,2012.75,31.0,14.0,11,1,0,6,1483
1,10,398,2013.25,24.0,15.0,5,2,0,2,1102
2,10,398,2013.75,14.0,21.0,4,2,0,1,950
3,10,398,2014.25,24.0,22.0,4,3,0,1,1270
4,10,398,2014.75,31.0,18.0,4,3,0,2,496


## 

## 3. Player valuation

In [26]:
player_valuations_df = valuation_date_to_year(player_valuations_df)[["player_id", "market_value_in_eur", "year", "date"]]
players_performance_value_df = pd.merge(player_performance_df, player_valuations_df, on=["player_id", "year"])


players_performance_value_df = players_performance_value_df.sort_values(by=['player_id', 'year', 'date'], ascending=[True, True, False])

# Fjern duplikater basert på 'player_id' og 'year', behold raden med nyeste dato
result_df = players_performance_value_df.drop_duplicates(subset=['player_id', 'year'], keep='first')
carlo_holse_rows = result_df[result_df['player_id'] == 345577]

print(carlo_holse_rows)
result_df.head()

        player_id  player_club_id     year  goals_for  goals_against  goals  \
125409     345577             190  2017.75       10.0           11.0      2   

        assists  red_cards  yellow_cards  minutes_played  market_value_in_eur  \
125409        0          0             0             297               250000   

              date  
125409  2017-12-18  


,player_id,player_club_id,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date
0,10,398,2012.75,31.0,14.0,11,1,0,6,1483,4000000,2013-01-14
7,26,16,2012.75,59.0,29.0,0,0,0,1,2430,4000000,2013-01-13
23,65,931,2012.75,20.0,29.0,6,4,0,1,1395,7000000,2013-01-28
29,77,506,2012.75,11.0,3.0,0,0,0,0,307,1000000,2012-12-18
30,80,27,2012.75,4.0,0.0,0,0,0,0,90,1000000,2013-01-13


## 4. Player Characteristic

In [15]:
players_characteristics_df = players_df[['player_id', 'name', 'country_of_birth', 'country_of_citizenship','date_of_birth', 'height_in_cm', 'sub_position']]
players_characteristics_df['country_of_birth'].fillna(players_characteristics_df['country_of_citizenship'], inplace=True)
players_characteristics_df.dropna(inplace=True)
players_characteristics_df.head()

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward
1,26,Roman Weidenfeller,Germany,Germany,1980-08-06,190.0,Goalkeeper
4,80,Tom Starke,East Germany (GDR),Germany,1981-03-18,194.0,Goalkeeper
7,132,Tomas Rosicky,CSSR,Czech Republic,1980-10-04,179.0,Attacking Midfield
8,162,Marc Ziegler,Germany,Germany,1976-06-13,193.0,Goalkeeper


## 5. Combine and Calculate Age

In [16]:
total_df = pd.merge(players_characteristics_df, players_performance_value_df, on='player_id', how='left')
total_df.dropna(inplace=True)

total_df['age_at_evaluation'] = total_df.apply(calculate_age, axis=1)
total_df.head()

,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,player_club_id,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date,age_at_evaluation
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,4000000.0,2013-01-14,35
1,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,2000000.0,2013-06-19,35
2,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,1000000.0,2014-01-07,36
3,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,1000000.0,2014-07-07,36
4,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,1000000.0,2015-01-07,37


## 6. Add league as feature

In [17]:
#df_player = pd.read_csv('/work/cleaned_data_28.csv', sep=",", encoding="UTF-8")
df_player = total_df
df_club = pd.read_csv('../DataSets/RawData/clubs.csv', sep=",", encoding="UTF-8")

df_player = df_player.rename(columns={'player_club_id': 'club_id'})
df_club = df_club[["club_id", "domestic_competition_id"]]

df = pd.merge(df_player, df_club, on='club_id', how="left")
df.head()


,player_id,name,country_of_birth,country_of_citizenship,date_of_birth,height_in_cm,sub_position,club_id,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,date,age_at_evaluation,domestic_competition_id
0,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,4000000.0,2013-01-14,35,IT1
1,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,2000000.0,2013-06-19,35,IT1
2,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,1000000.0,2014-01-07,36,IT1
3,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,1000000.0,2014-07-07,36,IT1
4,10,Miroslav Klose,Poland,Germany,1978-06-09,184.0,Centre-Forward,398.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,1000000.0,2015-01-07,37,IT1


## Normalize market value

In [18]:
df['log_market_value_base10'] = np.log10(df['market_value_in_eur'])
df.to_csv("../DataSets/EncodedData/cleaned_data_04_12.csv", index=False)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=039d1f4a-fb47-4bbf-8592-3573ff918cac' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>